In [1]:
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import keras.backend as K

In [2]:

from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from patchify import patchify, unpatchify
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, Softmax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5402528667074840145
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5707399168
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9472595390804454493
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
]


In [3]:
img_folder = 'pipeline/processed/training/images/'
# mask_folder = '../train/masks/_root/'

In [4]:
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
def img_patchify(img_dir, save_dir, patch_size):
    """Adds padding to all images in folder and patchifies them using the patchify library. 
    The patchified images get saved in the specified folder and returned in an array.

    Args:
        img_dir (str): Path to image folder 
        save_dir (str): Path to folder in which the patches should be saved
        patch_size (int): Size of patches that should be made 

    Returns:
        array: Array of patched images
    """
    img = []
    tifs = []

    for image_filename in os.listdir(img_dir):
        if image_filename.endswith((".png", ".tif")):
            im = cv2.imread(os.path.join(img_dir, image_filename))
            img_name, extension = os.path.splitext(image_filename)

            h, w = im.shape[:2]
            height_padding = ((h // patch_size) + 1) * patch_size - h
            width_padding = ((w // patch_size) + 1) * patch_size - w

            top_padding = height_padding // 2
            bottom_padding = height_padding - top_padding

            left_padding = width_padding // 2
            right_padding = width_padding - left_padding

            padded_image = cv2.copyMakeBorder(im, top_padding, bottom_padding, left_padding, right_padding, cv2.BORDER_CONSTANT, value=[0, 0, 0])

            channels = 3 if extension == ".png" else 1
            patches = patchify(padded_image, (patch_size, patch_size, channels), step=patch_size)
            patches = patches.reshape(-1, patch_size, patch_size, channels)

            for i, patch in enumerate(patches):
                output_filename = f"{img_name}{i}{extension}"
                cv2.imwrite(os.path.join(save_dir, output_filename), patch)

                if extension == ".png":
                    img.append(patch)
                elif extension == ".tif":
                    tifs.append(patch)

    return img, tifs


In [6]:
def f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = TP / (Positives+K.epsilon())
        return recall
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = TP / (Pred_Positives+K.epsilon())
        return precision
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
        total = K.sum(K.square(y_true),[1,2,3]) + K.sum(K.square(y_pred),[1,2,3])
        union = total - intersection
        return (intersection + K.epsilon()) / (union + K.epsilon())
    return K.mean(f(y_true, y_pred), axis=-1)

In [7]:
# copied from Task 7 
def padder(image, patch_size):
    """
    Adds padding to an image to make its dimensions divisible by a specified patch size.

    This function calculates the amount of padding needed for both the height and width of an image so that its dimensions become divisible by the given patch size. The padding is applied evenly to both sides of each dimension (top and bottom for height, left and right for width). If the padding amount is odd, one extra pixel is added to the bottom or right side. The padding color is set to black (0, 0, 0).

    Parameters:
    - image (numpy.ndarray): The input image as a NumPy array. Expected shape is (height, width, channels).
    - patch_size (int): The patch size to which the image dimensions should be divisible. It's applied to both height and width.

    Returns:
    - numpy.ndarray: The padded image as a NumPy array with the same number of channels as the input. Its dimensions are adjusted to be divisible by the specified patch size.

    Example:
    - padded_image = padder(cv2.imread('example.jpg'), 128)

    """
    h = image.shape[0]
    w = image.shape[1]
    height_padding = ((h // patch_size) + 1) * patch_size - h
    width_padding = ((w // patch_size) + 1) * patch_size - w

    top_padding = int(height_padding/2)
    bottom_padding = height_padding - top_padding

    left_padding = int(width_padding/2)
    right_padding = width_padding - left_padding

    padded_image = cv2.copyMakeBorder(image, top_padding, bottom_padding, left_padding, right_padding, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    return padded_image

In [8]:
# U-Net model copied from Task 7 Image segmentation with DL 

def unet_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    s = inputs

    # Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(patch_size, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    # Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
     
    # Adjust the number of output channels to match the number of classes
    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=inputs, outputs=outputs)
    
    return model


In [9]:
mask_folders = ['_root','_shoot','_occluded_root','_seed']

In [12]:
for mask_folder in mask_folders:
    img_folder = f'raw/train/images/'
    masks_folder = f'raw/train/masks/{mask_folder}/'
    patch_size = 256

    
    # Training images
    train_image_datagen = ImageDataGenerator(validation_split=0.2,rescale=1./255)
    train_image_generator = train_image_datagen.flow_from_directory(
        img_folder,
        target_size=(patch_size, patch_size),
        batch_size=16,
        class_mode=None,  # None since you don't want labels for images
        color_mode='rgb',
        seed=42,
        subset='training'  # specify the subset as 'training'
    )

    # Validation images
    validation_image_generator = train_image_datagen.flow_from_directory(
        img_folder,
        target_size=(patch_size, patch_size),
        batch_size=16,
        class_mode=None,  # None since you don't want labels for images
        color_mode='rgb',
        seed=42,
        subset='validation'  # specify the subset as 'validation'
    )

    # Training masks
    train_mask_datagen = ImageDataGenerator(validation_split=0.2)  # You can also use validation_split here
    train_mask_generator = train_mask_datagen.flow_from_directory(
        masks_folder,
        target_size=(patch_size, patch_size),
        batch_size=16,
        color_mode='grayscale',  # Grayscale for multiclass segmentation
        class_mode=None,
        seed=42,
        subset='training'  # specify the subset as 'training'
    )

    # Validation masks
    validation_mask_generator = train_mask_datagen.flow_from_directory(
        masks_folder,
        target_size=(patch_size, patch_size),
        batch_size=16,
        color_mode='grayscale',  # Grayscale for multiclass segmentation
        class_mode=None,
        seed=42,
        subset='validation'  # specify the subset as 'validation'
    )
    validation_image_generator.samples//16
    len(train_image_generator)
    print(mask_folder)
    train_generator = zip(train_image_generator, train_mask_generator)
    validation_generator = zip(validation_image_generator, validation_mask_generator)

    # Define the U-Net model
    input_shape = (256, 256, 3)
    num_classes = 1
    model = unet_model(input_shape, num_classes)
    optimizer=Adam()
    # Compile the model
    model.compile(optimizer = Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy', f1, iou])

    # # Print a summary of the model architecture
    # model.summary()

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, mode='min')


    # Define the directory to save the best models
    model_save_dir = '../best_models/'


    # Save the best model with a unique name for each folder
    model_checkpoint = ModelCheckpoint(os.path.join(model_save_dir, f'best_model{mask_folder}.h5'), save_best_only=True)

    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_image_generator),
        validation_data=validation_generator,
        validation_steps=validation_image_generator.samples // 16,
        epochs=100,
        callbacks=[early_stopping, model_checkpoint]
    )
    num_batches = validation_image_generator.samples // validation_image_generator.batch_size

    all_predictions = []
    all_labels = []

    

    for _ in range(num_batches):
        img_batch, mask_batch = next(validation_generator)
        preds = model.predict(img_batch)
        all_predictions.append(preds)
        all_labels.append(mask_batch)

    # Concatenate predictions and labels across batches
    predictions = np.concatenate(all_predictions, axis=0)
    labels = np.concatenate(all_labels, axis=0)

    # Calculate IoU for the entire validation set
    iou_value = iou(K.variable(labels), K.variable(predictions))
    iou_value = K.eval(iou_value)
    print("IoU:", iou_value)
    fig, axs = plt.subplots(1, 3, figsize=(18, 4))

Found 7648 images belonging to 1 classes.
Found 1911 images belonging to 1 classes.
Found 7648 images belonging to 1 classes.
Found 1911 images belonging to 1 classes.
_root
Epoch 1/100
478/478 [==============================] - 86s 151ms/step - loss: 0.0435 - accuracy: 0.9876 - f1: 0.1472 - iou: 0.0367 - val_loss: 0.0398 - val_accuracy: 0.9908 - val_f1: 0.3945 - val_iou: 0.0916
Epoch 2/100
478/478 [==============================] - 73s 153ms/step - loss: 0.0152 - accuracy: 0.9953 - f1: 0.5990 - iou: 0.1267 - val_loss: 0.0203 - val_accuracy: 0.9931 - val_f1: 0.5973 - val_iou: 0.1295
Epoch 3/100
478/478 [==============================] - 73s 153ms/step - loss: 0.0109 - accuracy: 0.9962 - f1: 0.6848 - iou: 0.1504 - val_loss: 0.0182 - val_accuracy: 0.9935 - val_f1: 0.6190 - val_iou: 0.1432
Epoch 4/100
478/478 [==============================] - 74s 155ms/step - loss: 0.0089 - accuracy: 0.9967 - f1: 0.7466 - iou: 0.1617 - val_loss: 0.0141 - val_accuracy: 0.9945 - val_f1: 0.7071 - val_iou: 0

KeyboardInterrupt: 